In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
%matplotlib inline

In [63]:
df_hotel = pd.read_csv('../data/Hotels_Properties_Citywide.csv')
df_hotel_clean = df_hotel.dropna(inplace=False)
df2 = df_hotel_clean[["PARID", "Borough", "Postcode"]]
df2['Borough'] = df2['Borough'].replace(['1', '2', '3', '4', '5'], 
['Manhattan', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island'])
df2['Borough'] = df2['Borough'].replace(['MANHATTAN', 'BRONX', 'BROOKLYN', 'QUEENS', 'STATEN IS'], 
['Manhattan', 'Bronx', 'Brooklyn', 'Queens', 'Staten Island'])
df2.rename(columns={'Postcode': 'zipcode', 'Borough': 'neighbourhood_group'}, inplace=True)

In [17]:
df2.head()

,PARID,neighbourhood_group,zipcode
0,1000080039,Manhattan,10004
1,1000080051,Manhattan,10004
2,1000100033,Manhattan,10004
3,1000110029,Manhattan,10004
4,1000161301,Manhattan,10282


In [103]:
df_zipcode = pd.read_csv('../data/neighbourhoods.csv')
df_zipcode.head()

,neighbourhood_group,neighbourhood,zipcode
0,Bronx,Allerton,10467
1,Bronx,Baychester,10469
2,Bronx,Belmont,10457
3,Bronx,Belmont,10458
4,Bronx,Bronx Park,10460


In [67]:
df_zipcode = df_zipcode.groupby(['zipcode','neighbourhood_group']).first().reset_index()
df3 = pd.merge(df2, df_zipcode, on=['zipcode','neighbourhood_group'], how='left')
df3['neighbourhood'] = df3['neighbourhood'].fillna('.')
df3.head()

,PARID,neighbourhood_group,zipcode,neighbourhood
0,1000080039,Manhattan,10004,Financial District
1,1000080051,Manhattan,10004,Financial District
2,1000100033,Manhattan,10004,Financial District
3,1000110029,Manhattan,10004,Financial District
4,1000161301,Manhattan,10282,Battery Park City


In [69]:
df3.shape

(5473, 4)

In [70]:
# to csv
df3.to_csv('../data/Hotel_with_neighbour.csv', index=False)

In [56]:
df_airbnb = pd.read_csv('../data/airbnb_open_data_full_clean.csv')
df_airbnb['airbnb_counts_per_neighbourhood'] = df_airbnb.groupby('neighbourhood')['neighbourhood'].transform('count')
df_airbnb.head()

,id,name,host_id,host_name,neighbourhood,neighbourhood_group,latitude,longitude,price,calculated_host_listings_count,host_listings_neighbourhood_count,host_listings_neighbourhood_group_count,has_fire_alarm,has_co_monitor,airbnb_counts_per_neighbourhood
0,2539,Clean & quiet apt home by the park,2787,John,Kensington,Brooklyn,40.64529,-73.97238,299.0,9,2,9,True,True,117
1,2595,Skylit Midtown Castle,2845,Jennifer,Midtown,Manhattan,40.75356,-73.98559,175.0,3,1,3,True,True,1701
2,5121,BlissArtsSpace!,7356,Garon,Bedford-Stuyvesant,Brooklyn,40.68535,-73.95512,60.0,2,2,2,False,False,2779
3,45910,Beautiful Queens Brownstone! - 5BR,204539,Mark,Ridgewood,Queens,40.70309,-73.89963,425.0,6,3,6,True,True,315
4,5136,"Spacious Brooklyn Duplex, Patio + Garden",7378,Rebecca,Sunset Park,Brooklyn,40.66265,-73.99454,275.0,1,1,1,True,True,297


In [81]:
df_airbnb_temp = df_airbnb[['neighbourhood', 'neighbourhood_group', 'airbnb_counts_per_neighbourhood']].copy()
df_airbnb_temp.drop_duplicates(subset='neighbourhood', keep="last", inplace=True)
df_airbnb_temp.reset_index(drop=True, inplace=True)
df_airbnb_temp.head()

,neighbourhood,neighbourhood_group,airbnb_counts_per_neighbourhood
0,Fort Wadsworth,Staten Island,1
1,New Dorp,Staten Island,1
2,Neponsit,Queens,2
3,Country Club,Bronx,1
4,Willowbrook,Staten Island,1


In [90]:
df_airbnb_temp.shape

(243, 3)

In [108]:
# to csv
df_airbnb_temp.to_csv('../data/airbnb_counts.csv', index=False)

check zipcode

In [109]:
df_air_temp = pd.merge(df_airbnb_temp, df_zipcode, on=['neighbourhood_group', 'neighbourhood'], how='outer')
df_air_temp['airbnb_counts_per_neighbourhood'] = df_air_temp['airbnb_counts_per_neighbourhood'].fillna(0)

In [110]:
# to csv
df_air_temp.to_csv('../data/999.csv', index=False)

In [96]:
df_air_temp.isnull().sum()

neighbourhood                        0
neighbourhood_group                  0
airbnb_counts_per_neighbourhood      0
zipcode                            118
dtype: int64

In [71]:
df4 = pd.read_csv('../data/Hotel_with_neighbour.csv')

In [72]:
df4['hotel_counts_per_neighbourhood'] = df4.groupby('neighbourhood')['neighbourhood'].transform('count')

In [73]:
df4.value_counts(['neighbourhood', 'hotel_counts_per_neighbourhood']).sort_index()

neighbourhood       hotel_counts_per_neighbourhood
.                   196                               196
Allerton            6                                   6
Arden Heights       6                                   6
Arrochar            6                                   6
Astoria             138                               138
                                                     ... 
Upper West Side     714                               714
Washington Heights  8                                   8
Whitestone          2                                   2
Williamsburg        12                                 12
Woodlawn            9                                   9
Length: 95, dtype: int64

In [74]:
df4.head()

,PARID,neighbourhood_group,zipcode,neighbourhood,hotel_counts_per_neighbourhood
0,1000080039,Manhattan,10004,Financial District,95
1,1000080051,Manhattan,10004,Financial District,95
2,1000100033,Manhattan,10004,Financial District,95
3,1000110029,Manhattan,10004,Financial District,95
4,1000161301,Manhattan,10282,Battery Park City,2


In [75]:
# to csv
df4.to_csv('../data/Hotel_with_neighbour_counts.csv', index=False)

In [76]:
df5 = pd.read_csv('../data/Hotel_with_neighbour_counts.csv')

In [77]:
df5 = df5[['neighbourhood', 'neighbourhood_group', 'zipcode', 'hotel_counts_per_neighbourhood']].copy()
df5.drop_duplicates(subset='neighbourhood', keep="last", inplace=True)
df5.reset_index(drop=True, inplace=True)
df5.head()

,neighbourhood,neighbourhood_group,zipcode,hotel_counts_per_neighbourhood
0,Battery Park City,Manhattan,10282,2
1,Civic Center,Manhattan,10007,8
2,Chinatown,Manhattan,10013,907
3,Greenwich Village,Manhattan,10014,48
4,Chelsea,Manhattan,10001,185


check zipcode

In [112]:
df6 = df_air_temp.merge(df5, on=['neighbourhood_group', 'neighbourhood', 'zipcode'], how='outer')
df6.head()

,neighbourhood,neighbourhood_group,airbnb_counts_per_neighbourhood,zipcode,hotel_counts_per_neighbourhood
0,Fort Wadsworth,Staten Island,1.0,10305,NaN
1,New Dorp,Staten Island,1.0,10306,NaN
2,Neponsit,Queens,2.0,11694,NaN
3,Country Club,Bronx,1.0,10465,2.0
4,Willowbrook,Staten Island,1.0,10314,NaN


In [113]:
df6['hotel_counts_per_neighbourhood'] = df6['hotel_counts_per_neighbourhood'].fillna(0)
df6['airbnb_counts_per_neighbourhood'] = df6['airbnb_counts_per_neighbourhood'].fillna(0)

In [114]:
df6.isnull().sum()

neighbourhood                      0
neighbourhood_group                0
airbnb_counts_per_neighbourhood    0
zipcode                            0
hotel_counts_per_neighbourhood     0
dtype: int64

In [115]:
df6.to_csv('../data/Hotel_with_neighbour_counts_Airbnb.csv', index=False)